In [1]:
import numpy as np
MOL_LST = np.load("mol_iter_all.npy", allow_pickle=True)

In [1]:
from rdkit.Chem import rdMolTransforms
import altair as alt
import copy
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdForceFieldHelpers
from rdkit.Chem import ChemicalForceFields
from rdkit.Chem import AllChem

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
mol=Chem.AddHs(Chem.MolFromSmiles('CCCOC1=CC=C(Cl)C(C)=C1'))
for i, a in enumerate(mol.GetAtoms()):
    a.SetAtomMapNum(i)
AllChem.EmbedMolecule(mol,randomSeed=10) 

m2=copy.deepcopy(mol)
mp2 = AllChem.MMFFGetMoleculeProperties(m2)
energy=[]
confid=0
angles=range(0,370,10)
for angle in angles:
    confid+=1
    ff2 = AllChem.MMFFGetMoleculeForceField(m2, mp2)
    ff2.MMFFAddTorsionConstraint(1,2,3,4, False, angle - .1, angle + .1, 100.0)
    ff2.Minimize()
    energy.append(ff2.CalcEnergy())

    xyz=ff2.Positions()
    new_conf = Chem.Conformer(mol.GetNumAtoms())
    for i in range(mol.GetNumAtoms()):
        new_conf.SetAtomPosition(i, (m2.GetConformer(-1).GetAtomPosition(i)))
    new_conf.SetId(confid)
    mol.AddConformer(new_conf)


dfrdkit = pd.DataFrame({'angle':angles, 'energy':energy})
alt.Chart(dfrdkit).mark_line(point=True,interpolate="natural").encode(
    alt.X('angle:Q',
        scale=alt.Scale(domain=[0,360,350])
    ),
    alt.Y('energy:Q',
        scale=alt.Scale(zero=False)
    )
).interactive()

alt.Chart(...)

In [4]:
mol=Chem.AddHs(Chem.MolFromSmiles('CCCOC1=CC=C(Cl)C(C)=C1'))
for i, a in enumerate(mol.GetAtoms()):
    a.SetAtomMapNum(i)
AllChem.EmbedMolecule(mol,randomSeed=10)
conformer=mol.GetConformer(0)
m2=copy.deepcopy(mol)
mp = AllChem.MMFFGetMoleculeProperties(m2)
mp.SetMMFFOopTerm(False)    # That's the critical bit here - switch off out of plane terms for MMFF
ffm = AllChem.MMFFGetMoleculeForceField(m2, mp)
energy=[]
confid=0
angles=range(0,370,10)
for angle in angles:
    confid+=1
    ff2 = AllChem.MMFFGetMoleculeForceField(m2, mp)
    ff2.MMFFAddTorsionConstraint(1,2,3,4, False, angle - .1, angle + .1, 5.0)
    ff2.MMFFAddTorsionConstraint(4,3,2,1, False, angle - .1, angle + .1, 5.0)
    ff2.Minimize()
    energy.append(ff2.CalcEnergy())
    xyz=ff2.Positions()
    new_conf = Chem.Conformer(mol.GetNumAtoms())
    for i in range(mol.GetNumAtoms()):
        new_conf.SetAtomPosition(i, (m2.GetConformer(-1).GetAtomPosition(i)))
    new_conf.SetId(confid)
    mol.AddConformer(new_conf)

dfrdkit = pd.DataFrame({'angle':angles, 'energy':energy})
alt.Chart(dfrdkit).mark_line(point=True,interpolate="natural").encode(
    alt.X('angle:Q',
        scale=alt.Scale(domain=[0,360,350])
    ),
    alt.Y('energy:Q',
        scale=alt.Scale(zero=False)
    )
).interactive()

alt.Chart(...)

In [5]:
from ipywidgets import interact, interactive, fixed
import py3Dmol
patt = Chem.MolFromSmarts('c1ccccc1');patt
match = mol.GetSubstructMatch(patt)
AllChem.AlignMolConformers(mol,atomIds=match)
def drawit(m,p,confId):
    mb = Chem.MolToMolBlock(m,confId=confId)
    p.removeAllModels()
    p.addModel(mb,'sdf')
    p.setStyle({'stick':{}})
    return p.show()
viewer = py3Dmol.view(width=500, height=500)
mb = Chem.MolToMolBlock(mol,confId=0)
viewer.addModel(mb,'sdf')
viewer.setStyle({'stick':{}})
viewer.zoomTo()
conformerIds=[conf.GetId() for conf in mol.GetConformers()]
interact(drawit, m=fixed(mol),p=fixed(viewer),confId=(0,mol.GetNumConformers()-1))

interactive(children=(IntSlider(value=18, description='confId', max=37), Output()), _dom_classes=('widget-inte…

<function __main__.drawit(m, p, confId)>

In [27]:
m = Chem.rdmolfiles.MolFromMolFile('test_mol.mol',removeHs = False)
mp = AllChem.MMFFGetMoleculeProperties(m)
globalFF = AllChem.MMFFGetMoleculeForceField(m, mp)
m2 = m;
energy = []
angles = list(range(-180,180,5))
for x in range(-180,180,5):
	ff2 = AllChem.MMFFGetMoleculeForceField(m2, mp)
	ff2.MMFFAddTorsionConstraint(0,1,2,3, False, x - .1, x + .1, 10.0)
	ff2.Minimize()
	energy.append(globalFF.CalcEnergy(ff2.Positions()))

dfrdkit = pd.DataFrame({'angle':angles, 'energy':energy})
alt.Chart(dfrdkit).mark_line(point=True,interpolate="natural").encode(
    alt.X('angle:Q',
        scale=alt.Scale(domain=[0,360,350])
    ),
    alt.Y('energy:Q',
        scale=alt.Scale(zero=False)
    )
).interactive()

alt.Chart(...)

5517.2431454411535